In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=0cac8af42aa686ef61b6da9c1ccef6901aa8e3264ab1795f9dc9296223b18fe8
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [3]:
from pyspark.sql import *
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import functions as sf
from pyspark.sql.functions import udf
from pyspark.sql import SQLContext
import pdb
import calendar
import datetime
from pyspark.sql.types import StringType
from pyspark.sql.functions import lit,col

In [4]:
def findDay(date):
    ref = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    born = datetime.datetime.strptime(date, '%d %m %Y').weekday()
    return ref[int(born)]

In [5]:
sc = SparkContext('local')
# spark = SparkSession(sc)
sql = SQLContext(sc)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
sample_udf = udf(lambda x: findDay(x), StringType())
#df = sql.read.csv("joined_date - Copy.csv", inferSchema = True, header = True)
df = sql.read.csv("/content/drive/MyDrive/DBMS_project_final/Source Code/1-Datasets/KCPD_Crime_Data/KCcrime2010To2018.csv", inferSchema = True, header = True)


In [ ]:
df.createTempView(name='kc_crimels')
#pdb.set_trace()

In [ ]:
#getting the number of crimes in 2010 count by month
df2 = sql.sql("select Reported_month, count(1) from kc_crimels where Reported_year = '2010' group by Reported_month order by Reported_month")
print(df2.head())
#getting the maximum row of year 2010
df2 = sql.sql("select y.mnth,y.num from (select Reported_month as mnth, count(1) as num from kc_crimels where Reported_year = '2010' group by Reported_month order by num DESC) as y limit 1")
print(df2.head())
#getting the maximum from each year
df2 = sql.sql("select y.yr,y.mnth,y.num from (select Reported_year as yr,Reported_month as mnth, count(1) as num from kc_crimels group by Reported_year, Reported_month) as y")

Row(Reported_month=1, count(1)=9611)
Row(mnth=10, num=11993)


In [ ]:
df2.createTempView(name='temp1')
df3 = sql.sql("select yr,mnth,num from temp1 where num = (select max(num) from temp1 as f where f.yr = temp1.yr) group by yr,mnth,num order by yr")
print(df2.head(5))

[Row(yr=2012, mnth=10, num=10934), Row(yr=2010, mnth=7, num=11651), Row(yr=2010, mnth=12, num=9869), Row(yr=2015, mnth=2, num=7845), Row(yr=2017, mnth=3, num=10806)]


In [ ]:
sql.sql('drop table if exists temp1')
#maximum crime by category in 2010
df2 = sql.sql("select y.dec,y.num from (select Description as dec, count(1) as num from kc_crimels where Reported_year = '2010' group by Description order by num DESC) as y limit 1")
print(df2.head(5))

#maximum crime by category each year by category
df2 = sql.sql("select y.yr,y.dec,y.num from (select Reported_year as yr,Description as dec, count(1) as num from kc_crimels group by Reported_year, Description) as y")
df2.createTempView(name='temp3')
df3 = sql.sql("select yr,dec,num from temp3 where num = (select max(num) from temp3 as f where f.yr = temp3.yr) group by yr,dec,num order by yr")
print(df3.head(5))

#count of specific crime in each year ordered by year
df2 = sql.sql("select y.yr,y.dec,y.num from (select Reported_year as yr,Description as dec, count(1) as num from kc_crimels where Description = 'Burglary - Residence' group by Reported_year, Description) as y")
print(df2.head(5))

[Row(dec='Burglary - Residence', num=13598)]
[Row(yr=2010, dec='Burglary - Residence', num=13598), Row(yr=2011, dec='Burglary - Residence', num=12218), Row(yr=2012, dec='Burglary - Residence', num=12797), Row(yr=2013, dec='Burglary - Residence', num=11746), Row(yr=2014, dec='Burglary - Residence', num=10372)]
[Row(yr=2013, dec='Burglary - Residence', num=11746), Row(yr=2017, dec='Burglary - Residence', num=8336), Row(yr=2015, dec='Burglary - Residence', num=9066), Row(yr=2012, dec='Burglary - Residence', num=12797), Row(yr=2010, dec='Burglary - Residence', num=13598)]


In [ ]:
#joined date column
df3 = df.withColumn('joined_date',sf.concat(sf.col('Reported_day'),sf.lit(' '), sf.col('Reported_month'),sf.lit(' '),sf.col('Reported_year')))
df4 = df3.withColumn("week_day",sample_udf(col("joined_date")))

In [ ]:
df4

DataFrame[Report_No: int, Reported_year: int, Reported_month: int, Reported_day: int, Reported_hour: int, Reported_minute: int, From_year: int, From_month: int, From_day: int, From_hour: int, From_minute: int, Offense: int, IBRS: string, Description: string, Beat: string, Address: string, City: string, Zip_Code: double, Rep_Dist: string, Area: string, DVFlag: string, Invl_No: int, Involvement: string, Race: string, Sex: string, Age: double, Firearm_Used_Flag: string, joined_date: string, week_day: string]

In [ ]:
df4.createTempView(name='kc_port')

In [ ]:
#maximum crime day of the week
df2 = sql.sql("select y.yr,y.wk,y.num from (select Reported_year as yr, week_day as wk, count(1) as num from kc_port group by Reported_year, week_day) as y")
df2.createTempView(name='temp4')
df3 = sql.sql("select yr,wk,num from temp4 where num = (select max(num) from temp4 as f where f.yr = temp4.yr) group by yr,wk,num order by yr")
print(df3.head(5))

[Row(yr=2010, wk='Monday', num=20448), Row(yr=2011, wk='Monday', num=18704), Row(yr=2012, wk='Monday', num=18865), Row(yr=2013, wk='Monday', num=18443), Row(yr=2014, wk='Monday', num=18144)]


In [ ]:
#maximum crime of each day of the year
df2 = sql.sql("select y.yr,y.wk,y.dec,y.num from (select Reported_year as yr,week_day as wk, Description as dec, count(1) as num from kc_port group by Reported_year, week_day,dec) as y")
df2.createTempView(name='temp5')
df3 = sql.sql("select yr,wk,dec,num from temp5 where num = (select max(num) from temp5 as f where f.yr = temp5.yr and f.wk = temp5.wk) group by yr,wk,dec,num order by yr")
print(df3.head(5))


[Row(yr=2010, wk='Tuesday', dec='Burglary - Residence', num=2159), Row(yr=2010, wk='Saturday', dec='Property Damage', num=1620), Row(yr=2010, wk='Sunday', dec='Property Damage', num=1816), Row(yr=2010, wk='Wednesday', dec='Burglary - Residence', num=2173), Row(yr=2010, wk='Thursday', dec='Burglary - Residence', num=2088)]
